In [ ]:
from urllib.parse import urlparse, urldefrag
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import json

In [8]:
urls = set()

## Read a LOG file that was generated using the below code in the browser
# n=$$('a');for(u in n)console.log(n[u].href)
# Clean each line and extract the defragmented URL
with open('msrit.edu-1679487368740.log', 'r') as file:
    for line in file:
        # perform operation on each line
        url = line.strip().split('VM75:1 ')
        if len(url) > 1:
            urls.add(urldefrag(url[1])[0])
print(len(urls))


115


In [64]:
msrit_urls = []
external_urls = []
for url in urls:
    if url.endswith("/") or url.endswith(".html"):
        msrit_urls.append(url)
    else:
        external_urls.append(url)
print(len(msrit_urls))
print(len(external_urls))


74
41


In [10]:
faculty_urls = []
for url in msrit_urls:
    if 'faculty-detail' in url:
        faculty_urls.append(url)
print(len(faculty_urls))

0


In [52]:
department_code = []
for url in msrit_urls:
    if 'department' in url:
        department_code.append( url.split('/')[-1].split('.html')[0])
print(department_code)

['maths', 'cse', 'te', 'physics', 'civil-engineering', 'mca', 'chemical-engineering', 'architecture', 'humanities', 'cse_cs', 'iem', 'certificate-programs', 'ai_ml', 'int-411', 'ai_ds', 'mba', 'ece', 'eie', 'cse_ai_ml', 'ise', 'biotechnology', 'eee', 'medical-engineering', 'me', 'chemistry']


In [60]:
## Visit each department's faculty page in a browser and extract the faculty URLs
## We use Selenium to automate the process

driver = webdriver.Chrome()

faculty_urls = {}
for department in department_code:
    faculty_urls[department] = set()
    dep_url = 'https://msrit.edu/department/faculty.html?dept='+department+'.html'
    driver.get(dep_url)
    page_html = driver.page_source
    soup = BeautifulSoup(page_html, 'html.parser')
    for link in soup.find_all('a'):
        if link.get('href') is None:
            continue
        if 'faculty-detail' in link.get('href'):
            faculty_urls[department].add(
                "https://msrit.edu/department/" + link.get('href'))
    print(department, len(faculty_urls[department]))

driver.quit()


maths 23
cse 29
te 12
physics 11
civil-engineering 29
mca 11
chemical-engineering 12
architecture 34
humanities 8
cse_cs 4
iem 13
certificate-programs 0
ai_ml 3
int-411 0
ai_ds 3
mba 13
ece 31
eie 11
cse_ai_ml 3
ise 27
biotechnology 16
eee 18
medical-engineering 10
me 39
chemistry 10


In [61]:
# convert set to list for each department in faculty_urls
for department in faculty_urls:
    faculty_urls[department] = sorted(list(faculty_urls[department]))

# save faculty urls to json file
with open('faculty_urls.json', 'w') as file:
    json.dump(faculty_urls, file)


In [63]:
# save msrit_urls to json file
with open('msrit_urls.json', 'w') as file:
    json.dump(msrit_urls, file)


In [65]:
# save external_urls to json file
with open('external_urls.json', 'w') as file:
    json.dump(external_urls, file)

Our Dataset now comprises URLs of 

- 370 Faculty Pages
- 1208 Files (Documents, PDFs, Brochures)
- 75 Subpages from msrit.edu